In [2]:
import kfp
from google.cloud import aiplatform
from kfp.v2 import dsl, compiler
from kfp.v2.dsl import component
from typing import NamedTuple

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='user-demo.json'

In [3]:
@component(base_image="python:3.9")
def product_name(text: str) -> str:
    return text

In [4]:
@component(packages_to_install=["emoji"])
def emoji(
    text: str,
) -> NamedTuple(
    "Outputs",
    [
        ("emoji_text", str), 
        ("emoji", str),
    ],
):
    import emoji

    emoji_text = text
    emoji_str = emoji.emojize(':' + emoji_text + ':', language='alias')
    print("output one: {}; output_two: {}".format(emoji_text, emoji_str))
    return (emoji_text, emoji_str)

In [5]:
@component
def build_sentence(
    product: str,
    emoji: str,
    emojitext: str
) -> str:
    print("We completed the pipeline, hooray!")
    end_str = product + " is "
    if len(emoji) > 0:
        end_str += emoji
    else:
        end_str += emojitext
    return(end_str)

In [6]:
@kfp.dsl.pipeline(
    name="intro", 
    description="intro",
    pipeline_root="gs://<bucket>/demo"
)

def main_pipeline(
    project: str = "<project_id>",
    gcp_region: str = "us-central1",
    text: str = "Vertex Pipelines",
    emoji_str: str = "sparkles"
):
    product_task = product_name(text)
    
    emoji_task = emoji(emoji_str)
    
    consumer_task = build_sentence(
        product_task.output,
        emoji_task.outputs["emoji"],
        emoji_task.outputs["emoji_text"],
    )


In [8]:
compiler.Compiler().compile(
    pipeline_func=main_pipeline,
    package_path="pipeline.json"
)

In [19]:
aiplatform.init(project="<project_id>", location="us-central1")

job = aiplatform.PipelineJob(
    display_name="pipeline de prueba",
    template_path="pipeline.json",
    enable_caching=False,
    project="<project_id>",
    location="us-central1"
)

print('submit pipeline job ...')
job.submit(service_account="dev-dp-ml-vertex@<project_id>.iam.gserviceaccount.com")

submit pipeline job ...
Creating PipelineJob
PipelineJob created. Resource name: projects/624205664083/locations/us-central1/pipelineJobs/intro-20240602010141
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/624205664083/locations/us-central1/pipelineJobs/intro-20240602010141')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/intro-20240602010141?project=624205664083
